In [1]:

import torch
import numpy as np
import pandas as pd
import random
from pprint import pprint
import matplotlib.pyplot as plt
from tournamentgym import TournamentEnv
import os
import random
import copy
from dataclasses import dataclass
import torch
from torch.utils.data import Dataset
from transformers import DecisionTransformerConfig, DecisionTransformerModel, Trainer, TrainingArguments
from stable_baselines3.common.evaluation import evaluate_policy

torch.cuda.is_available()

c:\Users\sebas\IDrive-Sync\MetaLocalLLC\RL-Bots\Tournament-gym\tournamentgym.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


True

In [2]:
from tensorboard.plugins.hparams import api as thp
import tensorflow as tf
from hyperopt  import hp, fmin , tpe

In [3]:
from imitation.data.rollout import TrajectoryAccumulator
from imitation.data.huggingface_utils import trajectories_to_dataset
# from imitation.data import serialize
from imitation.data.rollout import rollout_stats
from imitation.data import serialize
from imitation.algorithms import sqil,bc

In [4]:
MW='M'

team_data=pd.read_csv(f'Process_data/{MW}_pm_encoded.csv')
train_env=TournamentEnv(team_stats=team_data,
                     season=2022,
                     verbose=False,
                     loading_bar=False,
                     exclude_seasons=[2023])


test_env=TournamentEnv(team_stats=team_data,
                     season=2023,
                     verbose=False,
                     loading_bar=False,
                     )
torch.cuda.is_available()

True

In [5]:
demonstrations=serialize.load('demonstrations/Mens_demonstrations.pkl')
demonstrations

In [6]:

stats = rollout_stats(demonstrations)
print(stats["return_mean"])

281.613


In [3]:
demonstrations[0].infos

NameError: name 'demonstrations' is not defined

In [19]:
    
bc_trainer = bc.BC(
                    observation_space=train_env.observation_space,
                    action_space=train_env.action_space,
                    demonstrations=demonstrations,
                    rng = np.random.default_rng(0),
                    # device='cuda'
                )
bc_trainer.train(
    # n_epochs=1000,
                 n_batches=100,
                 log_interval=1,
                 progress_bar =True)


In [20]:
bc_trainer.train(
    # n_epochs=1000,
                 n_batches=100,
                 log_interval=1,
                 progress_bar =True)

  0%|          | 0/100 [00:00<?, ?batch/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA_gather)

In [ ]:
reward, _ = evaluate_policy(bc_trainer.policy, test_env, 10)
print("Reward:", reward)

In [ ]:
def _get_arch(x):
    options=dict(
                     net_arch=[x, x])
    return options
def _get_train_freq(x,ep):
    options=(x,ep)
    return options

search_space = { 
    "learning_rate": hp.uniform("learning_rate",0.001,  1.0),
    'exploration_fraction': hp.uniform('exploration_fraction',0.001,  1.0),
    "gamma": hp.uniform("gamma",0,1),
    "tau": hp.uniform("tau",0,1),
    'train_freq': _get_train_freq(hp.choice('train_steps',[c for c in range(2,250,2)]),
                                  hp.choice('train_episode',['step','episode'])
                                  ),
    'buffer_size':hp.randint("buffer_size",0,100),
    'batch_size':hp.randint("batch_size",0,100),
    "gradient_steps": hp.randint("gradient_steps",0,100),
    "policy_kwargs": _get_arch(hp.choice("policy_kwargs",[64,128,264])),

        }

In [ ]:
def objective(params):
    params["tensorboard_log"]='Tournament_hypers/'
    params['device']='cuda'
    sqil_trainer = sqil.SQIL(
                        venv=train_env,
                        demonstrations=demonstrations,
                        policy="MlpPolicy",
                        rl_kwargs=params
                        
                            )
    
    bc_trainer = bc.BC(
                        observation_space=train_env.observation_space,
                        action_space=train_env.action_space,
                        demonstrations=demonstrations,
                        rng = np.random.default_rng(0),
                    )
    bc_trainer.train(total_timesteps=70_000,tb_log_name='Tourny_maker')
    # sqil_trainer.train(total_timesteps=70_000,tb_log_name='Tourny_maker')
    obs,info=test_env.reset(season=2023)
    terminated=False
    while not terminated:
        action=sqil_trainer.policy.predict(obs)[0]
        next_obs, reward, terminated,truncated, info=test_env.step(action=action)
        
        obs=next_obs


    
    return -info['score']

In [ ]:
best = fmin(objective, search_space, algo=tpe.suggest, max_evals=100)
best["tensorboard_log"]='Tournament_hypers/'
best['device']='cuda'
best

In [ ]:
sqil_trainer = sqil.SQIL(
                        venv=copy.deepcopy(train_env),
                        demonstrations=demonstrations,
                        **best
                        )

In [ ]:
reward_before_training, _ = evaluate_policy(sqil_trainer.policy, sqil_trainer.venv, 2)
print("Reward before training:", reward_before_training)

sqil_trainer.train(total_timesteps=1_000_000,tb_log_name='Trader')
reward_after_training, _ = evaluate_policy(sqil_trainer.policy, sqil_trainer.venv, 2)
print("Reward after training:", reward_after_training)

In [ ]:
obs,info=test_env.reset(season=2023)
terminated=False
while not terminated:
    action=sqil_trainer.policy.predict(obs)[0]
    next_obs, reward, terminated,truncated, info=test_env.step(action=action,cheat=False)

    obs=next_obs